In [1]:
import pennylane as qml
from pennylane import numpy as pnp
import numpy as np
from qiskit.quantum_info import SparsePauliOp
from scipy.optimize import minimize, differential_evolution
import os
import json
from susy_qm import calculate_Hamiltonian

In [34]:
potential = 'QHO'
cutoff = 8
shots = None

In [35]:
#calculate Hamiltonian and expected eigenvalues
H = calculate_Hamiltonian(cutoff, potential)

eigenvalues, eigenvectors = np.linalg.eig(H)
min_index = np.argmin(eigenvalues)
min_eigenvalue = eigenvalues[min_index]
min_eigenvector = np.asarray(eigenvectors[:, min_index])

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

num_qubits

4

In [36]:
min_3_ev = eigenvalues.argsort()[:3]
eigenvector = np.asarray(eigenvectors[:, min_3_ev[0]])

In [37]:
operator_pool = []
for i in range(num_qubits):
    operator_pool.append(qml.Identity(i))
    #operator_pool.append(qml.PauliX(i))
    operator_pool.append(qml.PauliY(i))
    operator_pool.append(qml.PauliZ(i))

cz_pool = []
for control in range(num_qubits):
        for target in range(num_qubits):
            if control != target:
                cz_pool.append(qml.CZ(wires=[control, target]))


In [38]:
dev = qml.device("default.qubit", wires=num_qubits, shots=shots)
basis_state = [1,0,0,0]

@qml.qnode(dev)
def circuit(times, op_list, try_cz=False, cz_wires=None, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))

    param_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=times[param_index], n=trotter_steps)
            param_index += 1

    if try_cz:
        qml.CZ(wires=cz_wires)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [39]:
dev = qml.device("default.qubit", wires=2*num_qubits, shots=shots)
basis_state = [1,0,0,0]

@qml.qnode(dev)
def swap_test(prev_times, times, prev_op_list, op_list, basis_state):

    trotter_steps=1
    bs = (basis_state + basis_state)
    qml.BasisState(bs, wires=range(2*num_qubits))
   
    prev_param_index = 0
    for pop in prev_op_list:

        if type(pop) == qml.CZ:
            qml.CZ(wires=pop.wires)
        else:
            qml.ApproxTimeEvolution(pop, time=prev_times[prev_param_index], n=trotter_steps)
            prev_param_index += 1
    
    param_index  = 0
    for op in op_list:
        if type(op) == qml.CZ:
            w0 = op.wires[0] + num_qubits
            w1 = op.wires[1] + num_qubits
            wire_map = {op.wires[0]:w0, op.wires[1]:w1}
            op = op.map_wires(wire_map)
            qml.CZ(wires=op.wires)
        else:
            wire = op.wires[0] + num_qubits
            wire_map = {op.wires[0]: wire}
            op = op.map_wires(wire_map)
            qml.ApproxTimeEvolution(op, time=times[param_index], n=trotter_steps)  
            param_index += 1
    
    qml.Barrier()
    for i in range(num_qubits):
        qml.CNOT(wires=[i, i+num_qubits])    
        qml.Hadamard(wires=i)   

    prob = qml.probs(wires=range(2*num_qubits))

    return prob


In [41]:
def overlap(prev_times, times, prev_op_list, op_list, basis_state):

    probs = swap_test(prev_times, times, prev_op_list, op_list, basis_state)

    overlap = 0
    for idx, p in enumerate(probs):

        bitstring = format(idx, '0{}b'.format(2*num_qubits))

        counter_11 = 0
        for i in range(num_qubits):
            a = int(bitstring[i])
            b = int(bitstring[i+num_qubits])
            if (a == 1 and b == 1):
                counter_11 +=1

        overlap += p*(-1)**counter_11

    return overlap

In [42]:
def loss_f(params, op_list, prev_ops, prev_params, beta=2.0):

    energy = circuit(params, op_list)
    
    penalty = 0
    try:
        if len(prev_ops) != 0:
            for prev_param, prev_op in zip([prev_params], [prev_ops]):
                ol = overlap(prev_times=prev_param, times=params, prev_op_list=prev_op, op_list=op_list, basis_state=basis_state)
                penalty += (beta*ol)
    except Exception as e:
        print(e)
        print(params)
        raise

    return energy + (penalty)

In [43]:
@qml.qnode(dev)
def grad_circuit(times, operator_ham, op_list, op_params, trotter_steps=1):

    qml.BasisState(basis_state, wires=range(num_qubits))
    param_index = 0
    for op in op_list:
        if type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            qml.ApproxTimeEvolution(op, time=op_params[param_index], n=trotter_steps)
            param_index +=1

    qml.ApproxTimeEvolution(operator_ham, time=times, n=trotter_steps)

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


In [44]:
def compute_grad(operator, op_list, op_params):
    t = pnp.tensor(0.0, requires_grad=True)
    grad_fn = qml.grad(grad_circuit)
    grad = grad_fn(t, operator, op_list, op_params)
    return grad

In [48]:
num_steps = 3
num_levels = 3
prev_ops = []
prev_params = []

for j in range(num_levels):

    print("######################################")
    print(f"level: {j}")

    if prev_ops:
        #op_list = [o for ops in prev_ops.copy()[-1] for o in ops]
        #op_params = [p for params in prev_params.copy()[-1] for p in params]
        pop_list = prev_ops.copy()[-1]
        pop_params = prev_params.copy()[-1]
    else:
        pop_list = []
        pop_params = []

    op_list = pop_list.copy()

    for i in range(num_steps):

        print(f"step: {i}")

        grad_list = []

        for op in operator_pool:
            grad = compute_grad(op, pop_list, pop_params)
            grad_list.append(abs(grad))

        maxidx = np.argmax(grad_list)
        max_op = operator_pool[maxidx]

        print(f"Max op is {max_op}")

        #if type(max_op) == qml.Identity:
        #    print("Op is Identity... skipping")
        #    break

        #if (op_list and max_op != op_list[-1]) or (not op_list):
        #op_list = pop_list.copy() + [max_op]
        op_list.append(max_op)
    
        x0 = [np.random.rand()*2*np.pi for _ in range(len(op_list))]
        #print(op_list, pop_list, pop_params, x0)
        res = minimize(
                    loss_f,
                    x0=x0,
                    args=(op_list, pop_list, pop_params),
                    method= "COBYLA",
                    options= {'maxiter':10000, 'tol': 1e-8}
                )
        
        if i!=0: 
            pre_min_e = min_e
            pre_op_params = op_params

        min_e = res.fun
        op_params = res.x
        print(f"op_params: {op_params}")

        print(f"Min E: {min_e}")
        print(res.success)

        #else:
        #    print("Op same as previous... skipping")


        print("Testing CZ pool")
        cz_e = []
        for term in cz_pool:
            energy = circuit(op_params, op_list, try_cz=True, cz_wires=term.wires)
            cz_e.append(energy)

        min_cz_e = cz_e[np.argmin(cz_e)]
        min_cz_term = cz_pool[np.argmin(cz_e)]
        if min_cz_e < min_e:
            print(f"Adding {min_cz_term} reduces energy further")
            op_list.append(min_cz_term)
            min_e = min_cz_e
            print(f"Min E: {min_e}")
        
        if i!=0:
            if abs(pre_min_e - min_e) < 1e-8:
                print("gradient converged")
                op_list.pop()
                op_params = pre_op_params
                break

      
    prev_ops.append(op_list.copy())
    prev_params.append(op_params.copy())
    print(prev_ops)
    #print(prev_params)
        
    

######################################
level: 0
step: 0
Max op is I(0)
op_params: [5.10356622]
Min E: 0.0
True
Testing CZ pool
step: 1
Max op is I(0)
op_params: [5.30069434 2.13206216]
Min E: 0.0
True
Testing CZ pool
gradient converged
[[I(0)]]
######################################
level: 1
step: 0
Max op is I(0)
op_params: [4.86706754 4.23773373]
Min E: 1.9999999999999991
True
Testing CZ pool
Adding CZ(wires=[0, 1]) reduces energy further
Min E: 0.0
step: 1
Max op is I(0)
op_params: [5.26468739 0.52501021 5.00620492 0.68588618]
Min E: 1.9999999999999991
True
Testing CZ pool
Adding CZ(wires=[0, 1]) reduces energy further
Min E: 0.0
gradient converged
[[I(0)], [I(0), I(0), CZ(wires=[0, 1]), I(0)]]
######################################
level: 2
step: 0


IndexError: index 2 is out of bounds for axis 0 with size 2

In [46]:
op_list

[]

In [47]:
prev_ops

[[], [], []]

In [ ]:
pre_op_params

In [ ]:
prev_ops

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def final_circuit(params, op_list):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.Identity:
            continue
        elif type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.state()

In [ ]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
print(qml.draw(final_circuit)(x0, op_list))

In [ ]:
def overlap_function(params, op_list):

    params = pnp.tensor(params, requires_grad=True)
    ansatz_state = final_circuit(params, op_list)
    
    overlap = np.vdot(min_eigenvector, ansatz_state)
    cost = np.abs(overlap)**2  

    return (1 - cost)

In [ ]:
x0 = np.random.uniform(0, 2 * np.pi, size=len(op_list))
o_iters = 10000
o_tol=1e-8

print("Running for overlap")
overlap_res = minimize(
    overlap_function,
    x0,
    args=(op_list),
    method= "COBYLA",
    options= {'maxiter':o_iters, 'tol': o_tol}
)

In [ ]:
overlap_res.fun

In [ ]:
dev = qml.device("default.qubit", wires=num_qubits)
@qml.qnode(dev)
def energy_circuit(params, op_list):

    qml.BasisState(basis_state, wires=range(num_qubits))
    params_index = 0
    for op in op_list:
        if type(op) == qml.Identity:
            continue
        elif type(op) == qml.CZ:
            qml.CZ(wires=op.wires)
        else:
            pauli_string = qml.pauli.pauli_word_to_string(op)
            qml.PauliRot(params[params_index], pauli_string, wires=op.wires)
            params_index += 1

    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))

In [ ]:
x0 = overlap_res.x
x0 = op_params
energy_circuit(x0, op_list)

In [ ]:
data = {"potential": potential,
        "cutoff": cutoff,
        "optimizer": "DE",
        "num steps": num_steps,
        "basis_state": basis_state,
        "op_list": [str(o) for o in op_list],
        "overlap": overlap_res.fun,
        "hellinger": hf_res.fun
        }

In [ ]:
data

In [ ]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\ADAPT-VQE\Files\TimeEv\\"
os.makedirs(base_path, exist_ok=True)
path = base_path + "{}_{}.json".format(potential, cutoff)
with open(path, 'w') as json_file:
    json.dump(data, json_file, indent=4)